In [5]:
import numpy as np
import os

In [48]:
pathway = '/Volumes/Jiamu2/Pypeit1.3.0/2020feb26_pypeit_red/hgc7a_results/det48'
possible_paths = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(pathway)) for f in fn]

In [49]:
spec1d_files = []
spec1d_dict = {}
for file in possible_paths:
    if ('spec1d_' in file) and ('.txt' in file):
        spec1d_files.append(file)
        last_slash_ind = file.rfind('/')
        path = file[:last_slash_ind+1]
        if path not in spec1d_dict:
            spec1d_dict[path] = []
        spec1d_dict[path].append(file[last_slash_ind+1:])

In [50]:
for path in spec1d_dict: 
    spec1d_texts = spec1d_dict[path]
    spec1d_contents = {}
    for file in spec1d_texts: #loop over the different observations of the same mask on the same date
        spec1d_contents[file] = {}
        with open(f'{path}{file}','r') as f: #read each spec1d.txt file
            lines = f.readlines()
        labels = lines[0].strip().split('|') #get the labels
        for j in range(len(labels)):
            labels[j] = labels[j].strip()
        while '' in labels:
            labels.remove('')
        labels.append('det_num')
        data = []
        for line in lines[1:]: #get the data
            line = line.strip().split('|')
            while '' in line:
                line.remove('')
            for j in range(len(line)):
                line[j] = line[j].strip()
                if '-DET' in line[j]:
                    det_num = line[j].split('-')[-1]
                try:
                    line[j] = float(line[j])
                except:
                    pass
            line.append(det_num)
            data.append(line)
        #save the labels and data into a dictionary
        for j in range(len(labels)):
            if labels[j] not in spec1d_contents[file]:
                spec1d_contents[file][labels[j]] = []
            for row in data:
                spec1d_contents[file][labels[j]].append(row[j])
            spec1d_contents[file][labels[j]] = np.array(spec1d_contents[file][labels[j]])
            
    #match objext names and loop over files
    filenames = spec1d_contents.keys()
    det_a_list = []
    det_b_list = []
    txt_list = []
    name_list = []
    for j,file in enumerate(filenames):
        first_det = -1
        for k in range(len(spec1d_contents[file]['name'])-1):
            #if first_det == -1:
                #first_det = spec1d_contents[file]['det_num'][k]
            name_matches = []
            det_a = []
            det_b = []
            
            for kk in range(k+1,len(spec1d_contents[file]['name'])):
                if k == kk:
                    continue
                elif spec1d_contents[file]['det_num'][k] == spec1d_contents[file]['det_num'][kk]:
                    continue
                elif spec1d_contents[file]['objname'][k] != spec1d_contents[file]['objname'][kk]:
                    continue
                else:
                    name_matches.append(spec1d_contents[file]['objname'][kk])
                    det_a.append(spec1d_contents[file]['name'][k])
                    det_b.append(spec1d_contents[file]['name'][kk])
                    
            if len(name_matches) == 0:
                    continue
            elif len(name_matches) == 2:
                    continue
            name_matches = np.array(name_matches)
            matches = np.array([[file],[name_matches[0]],[det_a[0]],[det_b[0]]])
            det_a_list.append(det_a[0])
            det_b_list.append(det_b[0])
            name_list.append(name_matches[0])
            txt_list.append(file)
    
    final_matches = {}
    det_a_matches = []
    det_b_matches = []
    name_list_matches = []
    for i in range(len(name_list)):
        if name_list[i] not in final_matches:
            final_matches[name_list[i]] = [[txt_list[i],txt_list[i]],
                                            [det_a_list[i],det_b_list[i]]]
        else:
            final_matches[name_list[i]][0].extend([txt_list[i],txt_list[i]])
            final_matches[name_list[i]][1].extend([det_a_list[i],det_b_list[i]])
            
    outpath = f'{path}coadd1d_results/'
    if not os.path.isdir(outpath):
        os.makedirs(outpath)
    for star in final_matches.keys():
        #print(final_matches[star][0][i],final_matches[star][1][i])
        with open(f'{outpath}{star}_coadd1d.txt','w') as f:
            f.write("# User-defined coadding parameters\n")
            f.write("[coadd1d]\n")
            f.write(f"    coaddfile = '{star}_coadd1d.fits'\n")
            f.write("    flux_value = False\n\n")
            f.write("# Read in the data\n")
            f.write("coadd1d read\n")
            ##LOOP OVER THE FILES!
            for i in range(len(final_matches[star][0])):
                spec1d_filepath = final_matches[star][0][i].split('.txt')[0]+'.fits'
                f.write(f"    {spec1d_filepath} {final_matches[star][1][i]}\n")
                #f.write(f"    {spec1d_filepath} {det_b}\n")
            f.write("coadd1d end")